# Text Generation with Neural Networks

Import necessary packages for preprocessing, model building, etc. We follow the steps described in the theoretical part of this summer school as follows:

0. Define Reseach Goal
2. Retrieve Data
3. Prepare Data
4. Explore Data
5. Model Data
6. Present and automate Model

In [6]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

# 1. Retrieve Data

Load your data! You can pick up data from everywhere, such as plain text, HTML, source code, etc.
You can either automatically download with Keras get_file function or download it manually and import it in this notebook.

## Example Data Set
[trump.txt](https://raw.githubusercontent.com/harshilkamdar/trump-tweets/master/trump.txt)

In [7]:
path = #TODO Optional
text = #TODO

print('corpus length:', len(text))

#path = get_file('trump.txt', origin='https://raw.githubusercontent.com/harshilkamdar/trump-tweets/master/trump.txt')
#text = io.open(path, encoding='utf-8').read().lower()

print('corpus length:', len(text))



corpus length: 2264441


# 2. Prepare Data

As described in the theoretical part of this workshop we need to convert our text into a word embedding that can be processed by a (later) defined Neural Network. 


## 2.1. Create Classes 
The goal after this step is to have a variable which contains the distinct characters of the text. Characters can be letters, digits, punctions, new lines, spaces, etc.

### Example:
Let's assume we have the following text as input: "hallo. "

After the following step, we want to have all distinct characters, i.e.:

``[ "h", "a", "l", "o", ".", " " ] ``


In [4]:
# chars = sorted(list(set(text)))
chars = #TODO
print('total chars:', len(chars))

total chars: 6


## 2.2. Create Train Set

In the following section we need to create our test set based on our text. The idea is to map a sequence of characters to a class. In this case, a class is one of our distinct characters defined in the previous task. 

This means that a sequence of characters result into a next character.

### Example:
Our text is still "hallo. "
The training set should be defined as follows:

``
["ha"] --> "l"
["al"] --> "l"
["ll"] --> "o"
["lo"] --> "."
["o."] --> " " 
``

In [9]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 4
step = 1
sentences = []
next_chars = []

In [9]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 4


In [10]:
for idx in range(len(sentences)):
    print(sentences[idx], ":" , next_chars[idx])

 hal : l
hall : o
allo : .
llo. :  


In [11]:
sentences = ["hall", "allo", "lloh", "loha", "ohal"]
next_chars = ["o", "h", "a", "l", "l"]

In [6]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [12]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)))
y = np.zeros((len(sentences), len(chars)))
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [13]:
x.shape
# seq

(5, 4, 4)

In [14]:
chars[:]

['a', 'h', 'l', 'o']

In [15]:
x[0]

array([[ 0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.]])

In [16]:
y[0]

array([ 0.,  0.,  0.,  1.])

In [17]:
# input
print(y[0], "->")
print(len(x[0]))
print()
print(len(x))

[ 0.  0.  0.  1.] ->
4

5


In [18]:
# build the model: a LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
#model.add(Activation('relu'))
#model.add(LSTM(128))
#model.add(Activation('relu'))          
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
model.summary()

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               68096     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 516       
_________________________________________________________________
activation_1 (Activation)    (None, 4)                 0         
Total params: 68,612
Trainable params: 68,612
Non-trainable params: 0
_________________________________________________________________


In [19]:
#model = Sequential()
#model.add(LSTM(128, input_shape=(maxlen, len(chars))))
#model.add(Dense(len(chars)))
#model.add(Activation('softmax'))

#model.summary()
#optimizer = RMSprop(lr=0.01)
#model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [20]:
# DAS WIRD MITGEGEBEN111!!!!

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [21]:
#

In [22]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.1,]:
        print('----- diversity:', diversity)

        generated = ''
        #sentence = text[start_index: start_index + maxlen]
        sentence = "hall"
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [23]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [24]:
model.fit(x, y,
          batch_size=128,
          epochs=3,
          callbacks=[print_callback])

Epoch 1/3
5/5 [==============================] - 1s 265ms/step - loss: 1.3876

----- Generating text after Epoch: 0
----- diversity: 0.1
----- Generating with seed: "hall"
halllllllllllllllllhllllllllllllhlllllllholllllllllllhlllllllllollllllllllllllllallllllllllllllllllllllllllllllllllllllohllllllllllllhllllllllllllolllllllllllllllllllllllllllllllllllollllllllllllllllollllllllllllllllllllllalllllllhllllllllllllllholllllllllllllllllllllllllllllllllllllllllllllhlllllllohllllllllllllllllllllllhllllllllllllllloallllllllllllllllllalllllllllohllllllolllllllhlllllllllllllllll
Epoch 2/3
5/5 [==============================] - 0s 2ms/step - loss: 1.0912

----- Generating text after Epoch: 1
----- diversity: 0.1
----- Generating with seed: "hall"
hallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohallohal